In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import *
import os

In [4]:
os.getcwd()

'c:\\Users\\User\\AppData\\Local\\Programs\\Microsoft VS Code'